# Neural networks using tensorflow

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

import matplotlib.pyplot as plt
import netCDF4 as n
%matplotlib inline
from utils import train_test_split

In [2]:
from tflow_reg import nn_model

## Import data

In [3]:
path = "./files/"
filenames = ["specific_humidity_Europa_sp.nc", "relative_humidity_Europa_sp.nc", "pressure_Europa_sp.nc",  
             "temperature_Europa_sp.nc", "total_cloud_cover_Europa_sp.nc"]


cloud = n.Dataset(path + filenames[-1], "r")
relative = n.Dataset(path + filenames[1], "r")
specific = n.Dataset(path + filenames[0], "r")
pressure = n.Dataset(path + filenames[2], "r")
temperature = n.Dataset(path + filenames[3], "r")

my_files = {'cloud':cloud, 'relative':relative, 'specific':specific, 'pressure':pressure, 'temperature':temperature}


tcc = cloud.variables["tcc"]
rh = relative.variables["r"]
q = specific.variables["q"]
sp = pressure.variables["sp"]
t2m = temperature.variables["t2m"]

my_vars = {'tcc':tcc, 'rh':rh, 'q':q, 'sp':sp, 't2m':t2m}

n_days = 1
n_grid_boxes = tcc.shape[1]*tcc.shape[2]
X = np.empty((n_grid_boxes*n_days*4, 4))
y = np.empty((n_grid_boxes*tcc.shape[2]*n_days*4))



for t in range(int(n_days)):
    i = 0
    for key, value in my_vars.items():
        if key == 'tcc':
            y[n_grid_boxes*t:n_grid_boxes*(t+1)] = np.array(value[t, : , :]).flatten()
        elif key == 'rh' or key == 'q':
            X[n_grid_boxes*t:n_grid_boxes*(t+1), i] = np.array(value[t, 0, : , :]).flatten()
            i = i+1
        else:
            X[n_grid_boxes*t:n_grid_boxes*(t+1), i] = np.array(value[t, : , :]).flatten()
            i=i+1

### Standarize and split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, split_size=0.3)

Xn_train = (X_train - X_train.mean(axis=0))/X_train.std(axis=0)
yn_train = (y_train - y_train.mean())/y_train.std()

Xn_test = (X_test - X_train.mean(axis=0))/X_train.std(axis=0)
yn_test = (y_test - y_train.mean())/y_train.std()

In [22]:
y.std()

0.0430799946475558

In [23]:
type(Xn_train)

numpy.ndarray

In [24]:
Xn_train.shape

(13152, 4)

In [25]:
# Xn_train = tf.cast(Xn_train, tf.float32)  # convert to float 32 so that we can use tf.matmul

## Neural net model

Feed forward neural net with 2 layers. Activation function: Sigmoid for all layers.

TODO: write as function

### Run neural network model

In [46]:
npred = Xn_train.shape[1]
noutput = Xn_train.shape[0]
n_nodes = [npred, 10, 50, 1]

In [47]:
xs = tf.placeholder("float")
ys = tf.placeholder("float")

output = nn_model(xs, n_nodes)

### MSE

In [48]:
cost = tf.reduce_mean(tf.square(output-ys))

### R2

In [49]:
total_error = tf.reduce_sum(tf.square(tf.subtract(ys, tf.reduce_mean(ys))))

unexplained_error = tf.reduce_sum(tf.square(tf.subtract(ys, output)))

R_squared = tf.subtract(1.0, tf.divide(unexplained_error, total_error)) 

### Gradient descent

In [50]:
eta = 0.001
train = tf.train.GradientDescentOptimizer(eta).minimize(cost)

In [51]:
type(Xn_train)

numpy.ndarray

### Plot the data

In [ ]:
with tf.Session() as sess:
    # Initiate session and initialize all vaiables
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    #saver.restore(sess,'yahoo_dataset.ckpt')
    c_t = []
    c_test = []
    r2_t = []
    r2_test = []
    for i in range(100):
        for j in range(Xn_train.shape[0]):
            sess.run([cost,train],feed_dict=    {xs:Xn_train[j,:].reshape(1,npred), ys:yn_train[j]})
            # Run cost and train with each sample
        c_t.append(sess.run(cost, feed_dict={xs:Xn_train,ys:yn_train}))
        c_test.append(sess.run(cost, feed_dict={xs:Xn_test,ys:yn_test}))
        print('Epoch :',i,'Cost train:',c_t[-1], 'Cost test:',c_test[-1])
        print('Epoch :',i,'R2 train:',r2_t[-1], 'R2 test:',r2_test[-1])
        r2_t.append(sess.run(R_squared, feed_dict={xs:Xn_train,ys:yn_train}))
        r2_test.append(sess.run(R_squared, feed_dict={xs:Xn_test,ys:yn_test}))
    pred = sess.run(output, feed_dict={xs:Xn_test})
    # predict output of test data after training
    print('Cost :',sess.run(cost, feed_dict={xs:Xn_test,ys:yn_test}))
    #y_test = denormalize(df_test,y_test)
    #pred = denormalize(df_test,pred)
    #Denormalize data
    
    
    
    plt.plot(range(yn_test.shape[0]),yn_test,label="Original Data")
    plt.plot(range(yn_test.shape[0]),pred,label="Predicted Data")
    plt.legend(loc='best')
    plt.ylabel('Total Cloud Cover')
    plt.xlabel('Points')
    #plt.title('Total Cloud Cover')
    plt.savefig('results/data_t0_2layers_10_50_split03.png')
    plt.show()
    
    
    plt.subplot(1,2,1)
    plt.plot(range(len(c_t)), c_t,label="MSE train")
    plt.plot(range(len(c_test)), c_test,label="MSE test")
    plt.legend(loc='best')
    plt.ylabel('MSE')
    plt.xlabel('Epochs')
    #plt.title('Cost vs. epochs')
    plt.tight_layout()
    
    plt.subplot(1,2,2)
    plt.plot(range(len(r2_t)), r2_t,label="R2 train")
    plt.plot(range(len(r2_test)), r2_test,label="R2 test")
    plt.legend(loc='best')
    plt.ylabel('R2')
    plt.xlabel('Epochs')
    #plt.title('R2 vs. epochs')
    plt.tight_layout()
    plt.savefig('results/t0_2layers_10_50_split03.png')
    plt.show()
    
  

Epoch : 0 Cost train: 1.7214315 Cost test: 1.8372331
Epoch : 1 Cost train: 1.7385457 Cost test: 1.848332
Epoch : 2 Cost train: 1.7445581 Cost test: 1.8514938
Epoch : 3 Cost train: 1.7505206 Cost test: 1.8398935
Epoch : 4 Cost train: 1.7543066 Cost test: 1.8427784
Epoch : 5 Cost train: 1.7569467 Cost test: 1.843484
Epoch : 6 Cost train: 1.7588092 Cost test: 1.8450073
Epoch : 7 Cost train: 1.761165 Cost test: 1.8458194
Epoch : 8 Cost train: 1.7613595 Cost test: 1.8459971
Epoch : 9 Cost train: 1.7612786 Cost test: 1.8460565
Epoch : 10 Cost train: 1.7601812 Cost test: 1.8459514
Epoch : 11 Cost train: 1.7599947 Cost test: 1.84606
Epoch : 12 Cost train: 1.7597454 Cost test: 1.8458714
Epoch : 13 Cost train: 1.7599583 Cost test: 1.8461429
Epoch : 14 Cost train: 1.7597142 Cost test: 1.8464797
Epoch : 15 Cost train: 1.7599053 Cost test: 1.8466016
Epoch : 16 Cost train: 1.7600702 Cost test: 1.8462858
Epoch : 17 Cost train: 1.7593733 Cost test: 1.8464155


In [ ]:
output.get_shape().as_list()

In [45]:
yn_test[-10:-1]

array([-0.48529008, -0.48529008, -0.48529008, -0.48529008, -0.48529008,
       -0.48529008, -0.48529008, -0.48529008, -0.48529008])

In [ ]:
feed_dict={xs:Xn_train[j,:], ys:yn_train[j]}

In [ ]:
Xn_test.shape

In [ ]:
tf.reshape(Xn_train[j,:], [1, npred])

In [ ]:
n_nodes

In [ ]:
output = nn_model(Xn_train, n_nodes)

In [ ]:
output.get_shape().as_list()[1]